In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from core.config import *
from core.data import JsonlIO
from core.datasets import get_answer_objects
from core.utils import create_empty_dataarray

In [3]:
jsonlio = JsonlIO()

In [4]:
root_exp_path = Path('/home/olivieri/exp/data/private/prompts_data/by_model/LRASPP_MobileNet_V3/class-splitted/new_preds/gemma3:12b-it-qat')

In [5]:
exp_names = ['_test', 'test_']

In [6]:
img_idxs_set = set()
pos_classes_set = set()

for exp_name in exp_names:
    exp_path = root_exp_path/exp_name
    eval_objs = get_answer_objects(exp_path/'evals.jsonl', None, jsonlio, return_state=False)
    img_idxs_set |= {obj['img_idx'] for obj in eval_objs}
    pos_classes_set |= {obj['pos_class'] for obj in eval_objs}

In [7]:
data_da = create_empty_dataarray({
    'exp': exp_names,
    'img_idx': sorted(list(img_idxs_set)),
    'pos_class': sorted(list(pos_classes_set)),
    'content': ['pred', 'score', 'reason']
})
data_da.shape

(2, 6, 8, 3)

In [8]:
img_idxs_set = set()
pos_classes_set = set()

for exp_name in exp_names:
    exp_path = root_exp_path/exp_name
    eval_objs = get_answer_objects(exp_path/'evals.jsonl', None, jsonlio, return_state=False)
    for obj in eval_objs:
        data_da.loc[{'exp': exp_name, 'img_idx': obj['img_idx'], 'pos_class': obj['pos_class'], 'content': 'pred'}] = (obj['content']['pred'] == 'correct')
        data_da.loc[{'exp': exp_name, 'img_idx': obj['img_idx'], 'pos_class': obj['pos_class'], 'content': 'score'}] = obj['content']['score']
        data_da.loc[{'exp': exp_name, 'img_idx': obj['img_idx'], 'pos_class': obj['pos_class'], 'content': 'reason'}] = obj['content']['reason']

In [9]:
data_da.sel({'content': 'pred'})

<xarray.DataArray (exp: 2, img_idx: 6, pos_class: 8)> Size: 768B
array([[[True, None, None, None, False, None, None, None],
        [True, None, None, None, None, None, None, None],
        [None, None, None, None, None, None, None, True],
        [None, None, None, None, False, None, True, None],
        [None, True, None, None, None, None, None, None],
        [None, None, False, True, None, False, None, None]],

       [[True, None, None, None, True, None, None, None],
        [True, None, None, None, None, None, None, None],
        [None, None, None, None, None, None, None, True],
        [None, None, None, None, False, None, True, None],
        [None, False, None, None, None, None, None, None],
        [None, None, False, True, None, False, None, None]]], dtype=object)
Coordinates:
  * exp        (exp) <U5 40B '_test' 'test_'
  * img_idx    (img_idx) int64 48B 0 1 2 3 4 5
  * pos_class  (pos_class) int64 64B 1 4 9 12 15 18 19 20
    content    <U6 24B 'pred'

In [10]:
data_da.sel({'content': 'pred'}).mean(dim='pos_class', skipna=True).mean(dim='img_idx')

<xarray.DataArray (exp: 2)> Size: 16B
array([0.72222222, 0.63888889])
Coordinates:
  * exp      (exp) <U5 40B '_test' 'test_'
    content  <U6 24B 'pred'

In [11]:
data_da.sel({'content': 'score'}).mean(dim='pos_class', skipna=True).mean(dim='img_idx')

<xarray.DataArray (exp: 2)> Size: 16B
array([3.38888889, 3.30555556])
Coordinates:
  * exp      (exp) <U5 40B '_test' 'test_'
    content  <U6 24B 'score'